INtro / Objective here

Short summary of methods
Data description and citation


In [1]:
import os
import shutil
from glob import glob

import earthpy as et # Standard Earthpy library
import earthpy.earthexplorer as etee
import geopandas as gpd #Geopandas for geodataframes
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
chi_dir = os.path.join(data_dir, 'chicago neighborhoods')
ndvi_dir= os.path.join(data_dir, 'chicago-green-space', 'processed')

for a_dir in (chi_dir, ndvi_dir):
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

ModuleNotFoundError: No module named 'earthpy'

In [ ]:
# Load shapefile url
chi_path = os.path.join(chi_dir, 'chicago_neighboods.shp')
if not os.path.exists (chi_path):
    chi_url = ('https://data.cityofchicago.org/api/geospatial/'
                'bbvz-uum9?method=export&format=Shapefile')
    gpd.read_file(chi_url).to_file(chi_path)

chi_gdf = gpd.read_file(chi_path).set_index('pri_neigh')

# Use loc method to select neighborhoods of interest
neigh_gdf = (
    chi_gdf
    .loc[['Humboldt Park', 'Lincoln Park']]


neigh_gdf

In [ ]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    bbox = etee.BBox(*geometry.bounds)
    print(name)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label= name.lower().replace(' ', '-'), 
        bbox=bbox,
        start='2021-01-01', 
        end='2021-12-31',
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

# ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood_name-ndvi-stats.csv')
#   if os.path.exists(ndvi_stats_path):
#       print('Reading in NDVI statistics file...')
#       ndvi_stats_df = pd.read_csv(ndvi_stats_path)
#   else:
#       print('NDvI stats file does not exist...')
#       ndvi_stats_df = pd.DataFrame()
#   for neighborhood_name, details in chi_gdf.iterrows():
#       if neighborhood_name in ndvi_stats_df.index:
#           print('Neighborhood stats alread calculated. Skipping.')
#           continue
      
#       download = download_neighborhood_data(
#           neighborhood_name, details.geometry, '2921-01-01', '2021-12-31'
#       )

In [ ]:
def loads_and_merge_arrays(name):
    """
    Merge neighborhood dataarrays with more than one .tif file

    Parameters
    ==========
    name : str
      The name used to label the download

    Returns
    =======
    merge_da: rxr.DataArray
        Data array with all the merged dta
    """
    print(name)
    data_path = os.path.join(
        et.io.HOME,
        et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    print(data_path)

    tif_files = [os.path.join(data_path, file) 
                 for file in os.listdir(data_path) 
                 if file.endswith('.tif')]

    # Load each .tif file using rioxarray 
    chi_das = [rxr.open_rasterio(tif, masked=True) for tif in tif_files]
    merged_da = rxrmerge.merge_arrays(chi_das)
    return merged_da



In [ ]:
def calculate_ndvi_stats(gdf, da, stats_path, override = False):
    """
    Calculate NDVI from band data and write statistic spread to .csv


    Parameters
    ==========
    gdf : gpd.GeoDataFrame
        One row with the neighborhood name and boundary.
    da : rxr.DataArray
        Multispectral raster data
    stats_path : pathlike
        The path to the statistics file to save results
        
    """
    name = str(gdf.index[0])
    print(f'Neighborhood name:{name}')


    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df)==0
        print (f'Is Stats file empty? {file_is_empty}')

        if not file_is_empty:
            print(name)
            print(list(stats_df.neighborhood))
            print(name in  list(stats_df.neighborhood))
            if (name in stats_df.neighborhood) and (not override):
                print('Stats already calculated. Skipping...')
                return
    

    reprojected_gdf = ( gdf.to_crs(da.rio.crs))
    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

    mode = 'w' if file_is_empty else 'a'
    ndvi_da = (
        (da.sel(band=4) - naip_da.sel(band=1))
        /(da.sel(band=4) + naip_da.sel(band=1))
    )
    pd.DataFrame(dict(
    neighborhood =  [name],
    ndvi_min = [float(ndvi_da.min())],
    ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
    ndvi_50pctl=[np.nanpercentile(ndvi_da, 50)],
    ndvi_75pctl=[np.nanpercentile(ndvi_da, 75)],
    ndvi_max = [float(ndvi_da.max())],
    ndvi_mean=[float(ndvi_da.mean())])
).to_csv(stats_path, mode = mode, header=file_is_empty)
    


In [ ]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in chi_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI Statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(
            ndvi_stats_path, 
            index_col='neighborhood') 
    
    if neighborhood_name in ndvi_stats_df.index:
        print('Neighborhood stats already calculated. Moving on...')
        continue

    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
    merged_da = loads_and_merge_arrays(neighborhood_name)
    calculate_ndvi_stats(
        chi_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    
    shutil.rmtree(downloader.data_dir)